In [3]:

# !pip install -qU langchain-google-genai

In [1]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

/Users/ali/anaconda3/envs/graph_rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
    # other params...
)

In [3]:
def preload_content(file_name):
    """
    read the file to string
    """
    with open(file_name, 'r') as file:
        content = file.read()
        
    return content

In [4]:
file_name = './ali_cv_markdown.md'

In [5]:
cached_content = preload_content(file_name)

In [6]:
cached_content

'# Muhammad Ali Abbas (Ali)\n\n## Contact Information\n- **Phone**: +923171585452  \n- **Email**: maliabbas366@gmail.com  \n\n## Profile Summary\nI am Muhammad Ali Abbas (Ali), a **Silver Medalist** Machine Learning Engineer and researcher with expertise in **Speech AI**, **NLP**, and **Computer Vision**. Over the years, I have made significant contributions to AI research, publishing papers on diagnosing COVID-19 using **Swin Transformer**, predictive maintenance in motor faults, and pneumonia detection through hybrid deep learning methods. My contributions extend to AI-based histopathological diagnosis of lung and colon cancers and identifying health factors during pandemics using feature selection and neural networks.\n\nCurrently, as a **Lead Machine Learning Engineer** at **Idrak AI Ltd.**, I have led the development of innovative AI solutions, such as inbound and outbound multilingual calling bots, CFO agents for financial analysis, and generative AI agents. Leveraging cutting-ed

In [7]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [10]:
from langchain_core.messages import trim_messages

In [11]:
trimmer = trim_messages(strategy="last", max_tokens=5, token_counter=len)

In [12]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

In [13]:
def call_model(state: MessagesState):
    system_prompt = (
        "You are Ali. Machine Learning Engineer"
        "Here is data attached to you. You need to chat with people on behalf of Ali."
        "You will do helpful conversation. Respond politely"
        "Here is Ali's Information:"
        f"{cached_content}"
        "Your replies are conversational and concise."
        "Do not go more than 2 sentences in a response."
    )
    trimmed_messages = trimmer.invoke(state["messages"])
    messages = [SystemMessage(content=system_prompt)] + trimmed_messages
    response = model.invoke(messages)
    return {"messages": response}

In [14]:
# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [18]:
# how to run

In [19]:
app.invoke(
    {"messages": [HumanMessage(content="Hi, I am ASIF.")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='Hi, I am ASIF.', additional_kwargs={}, response_metadata={}, id='8cac8b6b-6cb2-4f3d-a440-ceff805bedf9'),
  AIMessage(content="Hi Asif, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-9d09d9a9-112c-457b-b69d-c7a9c38cbc96-0', usage_metadata={'input_tokens': 8107, 'output_tokens': 20, 'total_tokens': 8127, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='I think we have study togather in UOS. Did you did BS IT in 

In [20]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


class ConversationManager:
    def __init__(self, model_params, file_name):
        """
        Initialize the conversation manager with model parameters and content file.
        """
        self.model = ChatGoogleGenerativeAI(**model_params)
        self.cached_content = self.preload_content(file_name)
        self.trimmer = trim_messages(strategy="last", max_tokens=5, token_counter=len)
        self.workflow = self.setup_workflow()
        self.memory = MemorySaver()
        self.app = self.workflow.compile(checkpointer=self.memory)

    @staticmethod
    def preload_content(file_name):
        """
        Preload the content from the file.
        """
        with open(file_name, 'r') as file:
            content = file.read()
        return content

    def setup_workflow(self):
        """
        Set up the workflow graph.
        """
        def call_model(state: MessagesState):
            system_prompt = (
                "You are Ali. Machine Learning Engineer."
                "Here is data attached to you. You need to chat with people on behalf of Ali."
                "You will do helpful conversation. Respond politely."
                "Here is Ali's Information:"
                f"{self.cached_content}"
                "Your replies are conversational and concise."
                "Do not go more than 2 sentences in a response."
            )
            trimmed_messages = self.trimmer.invoke(state["messages"])
            messages = [SystemMessage(content=system_prompt)] + trimmed_messages
            response = self.model.invoke(messages)
            return {"messages": response}

        workflow = StateGraph(state_schema=MessagesState)
        workflow.add_node("model", call_model)
        workflow.add_edge(START, "model")
        return workflow

    def handle_conversation(self, thread_id, user_message):
        """
        Handle a conversation for the given thread_id.
        """
        response = self.app.invoke(
            {"messages": [HumanMessage(content=user_message)]},
            config={"configurable": {"thread_id": thread_id}},
        )
        return response


# Example Usage
if __name__ == "__main__":
    model_params = {
        "model": "gemini-2.0-flash-exp",
        "temperature": 0.2,
        "max_tokens": None,
        "timeout": None,
        "max_retries": 2,
        "safety_settings": {
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },
    }
    file_name = './ali_cv_markdown.md'

    manager = ConversationManager(model_params, file_name)

    # Simulate a conversation
    thread_id = "1"
    user_message = "Hi, I am Asif."
    response = manager.handle_conversation(thread_id, user_message)

    print(response)


{'messages': [HumanMessage(content='Hi, I am Asif.', additional_kwargs={}, response_metadata={}, id='bc5ad733-5c69-4cb6-ba77-759be7148ba3'), AIMessage(content="Hi Asif, it's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-7c7639f4-ce9b-48f2-bc69-601b979a1f1e-0', usage_metadata={'input_tokens': 8109, 'output_tokens': 20, 'total_tokens': 8129, 'input_token_details': {'cache_read': 0}})]}


In [21]:
model_params = {
        "model": "gemini-2.0-flash-exp",
        "temperature": 0.2,
        "max_tokens": None,
        "timeout": None,
        "max_retries": 2,
        "safety_settings": {
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },
    }
file_name = './ali_cv_markdown.md'

manager = ConversationManager(model_params, file_name)

# Simulate a conversation
thread_id = "1"
user_message = "Hi, I am Asif."
response = manager.handle_conversation(thread_id, user_message)

print(response)

{'messages': [HumanMessage(content='Hi, I am Asif.', additional_kwargs={}, response_metadata={}, id='5cf5be10-927f-4b4f-9613-5a6084b50c42'), AIMessage(content="Hi Asif, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-34c1b436-5cd7-4fbe-a35d-c94ba0f85b71-0', usage_metadata={'input_tokens': 8109, 'output_tokens': 20, 'total_tokens': 8129, 'input_token_details': {'cache_read': 0}})]}


In [22]:
user_message = "When u completed bs"
response = manager.handle_conversation(thread_id, user_message)

print(response)

{'messages': [HumanMessage(content='Hi, I am Asif.', additional_kwargs={}, response_metadata={}, id='5cf5be10-927f-4b4f-9613-5a6084b50c42'), AIMessage(content="Hi Asif, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-34c1b436-5cd7-4fbe-a35d-c94ba0f85b71-0', usage_metadata={'input_tokens': 8109, 'output_tokens': 20, 'total_tokens': 8129, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='When u completed bs', additional_kwargs={}, response_metadata={}

In [23]:
user_message = "How many students were there?"
response = manager.handle_conversation(thread_id, user_message)

print(response)

{'messages': [HumanMessage(content='Hi, I am Asif.', additional_kwargs={}, response_metadata={}, id='5cf5be10-927f-4b4f-9613-5a6084b50c42'), AIMessage(content="Hi Asif, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-34c1b436-5cd7-4fbe-a35d-c94ba0f85b71-0', usage_metadata={'input_tokens': 8109, 'output_tokens': 20, 'total_tokens': 8129, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='When u completed bs', additional_kwargs={}, response_metadata={}